In [1]:
import torch
from torch2trt import TRTModule

OPTIMIZED_MODEL = 'resnet18_baseline_att_224x224_A_epoch_249_trt.pth'

model_trt = TRTModule()
model_trt.load_state_dict(torch.load(OPTIMIZED_MODEL))

<All keys matched successfully>

In [2]:
WIDTH = 224
HEIGHT = 224

#data = torch.zeros((1, 3, HEIGHT, WIDTH)).cuda()
#print(data)

In [3]:
import json
import trt_pose.coco

with open('human_pose.json', 'r') as f:
    human_pose = json.load(f)

topology = trt_pose.coco.coco_category_to_topology(human_pose)
#print(topology)

In [4]:
import cv2
import torchvision.transforms as transforms
import PIL.Image

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()
device = torch.device('cuda')

def preprocess(image):
    global device
    device = torch.device('cuda')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [5]:
from trt_pose.draw_objects import DrawObjects
from trt_pose.parse_objects import ParseObjects

parse_objects = ParseObjects(topology)
draw_objects = DrawObjects(topology)

In [6]:
def execute(change):
    image = change
    data = preprocess(image)
    cmap, paf = model_trt(data)
    cmap, paf = cmap.detach().cpu(), paf.detach().cpu()
    counts, objects, peaks = parse_objects(cmap, paf)#, cmap_threshold=0.15, link_threshold=0.15)
    draw_objects(image, counts, objects, peaks)
    co = getcoo(topology,image,counts,objects,peaks)
   # print(co)
    #image_w.value = bgr8_to_jpeg(image[:, ::-1, :])
    return co

In [7]:
import cv2
import numpy as np
np.set_printoptions(threshold=np.inf)

def getcoo(topology, image, object_counts, objects, normalized_peaks):
    height = image.shape[0]
    width = image.shape[1]
    
    K = topology.shape[0]
    count = int(object_counts[0])
    K = topology.shape[0]
    #print(count)
    #print(objects)
    #print(normalized_peaks)
    #print("1")
    co_all =[]
    for i in range(count):
        color = (0, 255, 0)
        obj = objects[0][i]
        C = obj.shape[0]
        #print(obj)
        #print(C)
        coordinate = []
        for j in range(C):
            k = int(obj[j])
            if k >= 0:
                peak = normalized_peaks[0][j][k]
                x = round(float(peak[1]) * width)
                y = round(float(peak[0]) * height)
                #cv2.circle(image, (x, y), 3, color, 2)
                a = [x,y]
                #print(a)
                coordinate.append(a)
            else:
                a = [0,0]
                coordinate.append(a)
        co_all.append(coordinate)
    return co_all

In [9]:
camera.cap.release()

In [8]:
#important!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!backup
# just estimate the skeleton and show it out by opencv

from jetcam.usb_camera import USBCamera
# from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg
import cv2
import socket
import time

camera = USBCamera(width=WIDTH, height=HEIGHT, capture_fps=30)
# camera = CSICamera(width=WIDTH, height=HEIGHT, capture_fps=30)
while True:
    torch.cuda.current_stream().synchronize()
    st=time.time()
    image =camera.read()
   
    co=execute(image) 
    cv2.imshow("ds", image)
    print(co)
    #print(co[0][17][0])
    torch.cuda.current_stream().synchronize()
    end=time.time()
    #print(end-st)
    if (cv2.waitKey(1) & 0xFF) == ord('q'):
        break
#camera.running = True
cap.release()
cv2.destroyAllWindows()
#image1 =camera.read()
#def callback(change):
  #  image = 
#print(image)
#print(image1)

[[[107, 113], [117, 99], [98, 106], [141, 107], [92, 113], [165, 170], [91, 167], [0, 0], [59, 218], [0, 0], [89, 151], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [129, 169]]]
[[[108, 112], [119, 99], [100, 105], [142, 109], [94, 115], [166, 170], [91, 167], [0, 0], [62, 215], [0, 0], [86, 157], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [130, 170]]]
[[[109, 113], [119, 99], [101, 105], [142, 109], [95, 114], [166, 171], [91, 169], [0, 0], [60, 219], [0, 0], [85, 161], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [129, 170]]]
[[[112, 112], [122, 100], [105, 105], [144, 108], [99, 113], [169, 171], [94, 170], [0, 0], [64, 220], [0, 0], [90, 158], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [132, 171]]]
[[[121, 111], [130, 98], [112, 103], [149, 108], [105, 116], [174, 174], [99, 174], [0, 0], [65, 220], [0, 0], [90, 159], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [138, 173]]]
[[[132, 100], [142, 88], [124, 91], [161, 102], [118, 105], [190, 157], [117, 183], [0, 0

[[[147, 94], [158, 83], [140, 87], [177, 93], [0, 0], [0, 0], [123, 172], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [167, 170]], [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [197, 67], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]], [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [102, 139], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]]
[[[147, 94], [157, 83], [140, 87], [175, 94], [0, 0], [207, 165], [123, 174], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [166, 170]], [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [198, 67], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]], [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [102, 139], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]]
[[[147, 94], [157, 82], [140, 86], [176, 94], [0, 0], [207, 167]

[[[165, 97], [174, 86], [156, 91], [193, 93], [151, 106], [220, 164], [139, 165], [0, 0], [79, 198], [0, 0], [112, 122], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [179, 166]], [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [210, 85], [0, 0], [202, 61], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]]
[[[157, 97], [166, 86], [149, 92], [187, 94], [0, 0], [215, 162], [130, 166], [0, 0], [74, 206], [0, 0], [100, 137], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [174, 165]], [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [191, 62], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]], [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [102, 126], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]]
[[[140, 102], [151, 88], [132, 97], [174, 94], [0, 0], [203, 158], [122, 175], [0, 0], [67, 213], [0, 0], [89, 140], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [163, 167]], [[0, 0], [0

[[[78, 66], [93, 49], [70, 55], [127, 63], [0, 0], [171, 137], [65, 166], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [119, 153]]]
[[[72, 64], [88, 47], [66, 55], [123, 63], [0, 0], [169, 139], [63, 167], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [117, 154]]]
[[[72, 63], [87, 47], [65, 55], [122, 63], [0, 0], [167, 139], [63, 167], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [116, 154]]]
[[[72, 62], [87, 46], [64, 54], [120, 65], [0, 0], [166, 138], [62, 167], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [115, 154]]]
[[[72, 62], [87, 46], [64, 54], [120, 65], [0, 0], [166, 139], [62, 169], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [114, 155]]]
[[[73, 61], [88, 46], [64, 53], [120, 67], [0, 0], [166, 139], [62, 169], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [115, 155]]

NameError: name 'cap' is not defined

In [17]:
#important! detect and send skeleton data to MACbook of liwangyuhao

from jetcam.usb_camera import USBCamera
# from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg
import cv2
import socket
import time

soc = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
soc.connect(('192.168.100.82', 8814))

camera = USBCamera(width=WIDTH, height=HEIGHT, capture_fps=30)
# camera = CSICamera(width=WIDTH, height=HEIGHT, capture_fps=30)
while True:
    t1=time.time()
    image =camera.read()
   
    co = execute(image) 
    cv2.imshow("ds", image)
    #str_co = ",".join
    str_co = ",".join(map(str, co))
    #print(str_co)
    strco=str_co+"#"
    print(strco)
    soc.send(strco.encode())
    
    if (cv2.waitKey(1) & 0xFF) == ord('q'):
        soc.send(b'exit')
        soc.close()
        break
    t2=time.time()
    print(t2-t1)
#camera.running = True
cap.release()
cv2.destroyAllWindows()
#image1 =camera.read()
#def callback(change):
  #  image = 
#print(image)
#print(image1)

[[29, 98], [42, 84], [24, 88], [74, 100], [0, 0], [114, 166], [27, 177], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [71, 171]]#
0.18962764739990234
[[28, 98], [42, 83], [24, 88], [74, 100], [0, 0], [114, 167], [27, 177], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [71, 172]]#
0.09435892105102539
[[28, 99], [42, 84], [24, 89], [74, 100], [0, 0], [114, 167], [27, 177], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [71, 172]]#
0.09897041320800781
[[28, 99], [42, 84], [24, 89], [74, 100], [0, 0], [114, 169], [26, 177], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [71, 173]],[[0, 0], [0, 0], [0, 0], [74, 100], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]#
0.10515856742858887
[[28, 99], [42, 84], [24, 89], [74, 100], [0, 0], [114, 169], [25, 177], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0],

[[52, 110], [65, 94], [41, 96], [89, 96], [33, 102], [118, 174], [25, 178], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [72, 175]],[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [128, 86], [0, 0], [117, 77], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]#
0.09163641929626465
[[53, 110], [65, 94], [41, 96], [89, 96], [33, 101], [118, 175], [24, 178], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [72, 175]],[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [118, 75], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]#
0.08795833587646484
[[53, 111], [66, 94], [42, 96], [90, 95], [33, 101], [118, 174], [24, 178], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [72, 175]],[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [123, 78], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]#
0.0876500

[[50, 105], [61, 91], [42, 94], [82, 100], [35, 102], [114, 169], [25, 185], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [70, 175]]#
0.08572745323181152
[[51, 108], [61, 95], [42, 97], [83, 102], [35, 103], [114, 169], [26, 186], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [70, 176]]#
0.08468055725097656
[[51, 110], [61, 96], [42, 98], [83, 103], [35, 102], [114, 169], [26, 186], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [70, 176]]#
0.08491158485412598
[[51, 110], [61, 97], [42, 98], [83, 104], [36, 103], [113, 169], [26, 185], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [70, 177]]#
0.08870267868041992
[[51, 109], [62, 96], [42, 97], [83, 103], [36, 102], [113, 167], [26, 184], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [70, 175]]#
0.08467411994934082
[[51, 109], [62, 96], [43, 97], [83, 103], [36, 101], [

[[41, 119], [52, 104], [32, 106], [81, 107], [0, 0], [114, 175], [23, 179], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [69, 176]]#
0.08739423751831055
[[41, 116], [54, 101], [33, 104], [82, 107], [0, 0], [114, 175], [23, 181], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [70, 177]]#
0.08459019660949707
[[42, 114], [54, 99], [34, 101], [82, 107], [0, 0], [114, 175], [25, 179], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [70, 177]]#
0.085418701171875
[[43, 113], [56, 97], [35, 101], [83, 107], [0, 0], [115, 175], [25, 179], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [71, 176]]#
0.08623170852661133
[[44, 111], [57, 97], [36, 100], [83, 107], [0, 0], [115, 175], [25, 179], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [70, 177]]#
0.08334827423095703
[[45, 112], [57, 97], [36, 100], [83, 107], [0, 0], [116, 175], [

[[47, 120], [59, 104], [38, 106], [85, 105], [0, 0], [117, 177], [23, 181], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [71, 177]]#
0.08455538749694824
[[47, 121], [59, 105], [37, 107], [85, 104], [0, 0], [117, 177], [23, 181], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [71, 177]]#
0.08653759956359863
[[47, 120], [59, 104], [37, 107], [85, 105], [0, 0], [117, 177], [25, 179], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [71, 177]]#
0.08438968658447266
[[47, 120], [59, 104], [37, 107], [85, 105], [0, 0], [117, 177], [25, 179], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [71, 177]]#
0.08462119102478027
[[47, 120], [59, 104], [37, 107], [85, 104], [0, 0], [117, 177], [25, 180], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [71, 177]],[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [24, 181], [0, 0], [0, 0], [0, 0

...

[[0, 0], [0, 0], [0, 0], [15, 97], [0, 0], [43, 174], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [5, 173]]#
0.08246970176696777
[[0, 0], [0, 0], [0, 0], [14, 98], [0, 0], [42, 174], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [2, 172]]#
0.07918000221252441
[[0, 0], [0, 0], [0, 0], [14, 98], [0, 0], [42, 174], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [5, 171]]#
0.0799715518951416
[[0, 0], [0, 0], [0, 0], [11, 94], [0, 0], [39, 175], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]#
0.0832219123840332
[[0, 0], [0, 0], [0, 0], [10, 94], [0, 0], [38, 175], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [2, 174]]#
0.08317303657531738
[[0, 0], [0, 0], [0, 0], [9, 98], [0, 0], [34, 174], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]

[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [29, 170], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]#
0.08026909828186035
[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [29, 170], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]#
0.07920479774475098
[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [30, 170], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]#
0.07932853698730469
[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [29, 170], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]#
0.0811760425567627
[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [29, 170], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]#
0.08073663711547852
[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [29, 170], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]#
0.07567

[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [27, 171], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]#
0.08160090446472168
[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [29, 171], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]#
0.08017110824584961
[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [29, 171], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]#
0.07912468910217285
[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [27, 171], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]#
0.07891392707824707
[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [29, 171], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]#
0.08091020584106445
[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [27, 171], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]#
0.0809

#
0.0746157169342041
[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [14, 151], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]#
0.08371376991271973
[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [14, 151], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]#
0.07546186447143555
[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [14, 151], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]#
0.08060073852539062
[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [14, 151], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]#
0.0802469253540039
#
0.07468390464782715
[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [15, 151], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]#
0.08086013793945312
[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [16, 151], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0

BrokenPipeError: [Errno 32] 断开的管道

In [17]:
camera.cap.release()

In [16]:
#important!modify servo's direction by recieved data from fisheye camera and then send it to MAC

from jetcam.usb_camera import USBCamera
# from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg
import cv2
import socket
import time
import Jetson.GPIO as GPIO
import threading
local_ip = '192.168.100.218'

local_port = 9924
position = 10


soc = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
soc.connect(('192.168.100.82', 8825))
            
def udp_receiver(ip, port):
    # 创建UDP套接字
    udp_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    # 绑定地址和端口
    bind_address = (ip, port)
    udp_socket.bind(bind_address)
    global position
    try:
        while True:
            # 接收数据
            data, addr = udp_socket.recvfrom(1024)
            position = float(data.decode('utf-8'))
            print(type(position))
            print(f"接收到来自 {addr} 的数据: {data.decode('utf-8')}")
    except KeyboardInterrupt:
        print("接收程序终止.")
    finally:
        # 关闭套接字
        udp_socket.close()
        
treading1 = threading.Thread(target=udp_receiver,args=(local_ip,local_port))

treading1.start()

# 设置GPIO引脚编号方式
GPIO.setmode(GPIO.BOARD)

# 定义舵机连接的GPIO引脚
servo_pin = 33

# 设置GPIO引脚为输出模式
GPIO.setup(servo_pin, GPIO.OUT,initial=GPIO.HIGH)

# 创建PWM对象
pwm = GPIO.PWM(servo_pin, 50)  # 使用50Hz的PWM频率
pwm.start(2.5)

def angleToDutyCycle(angle):
    return 2.5 + angle / 180.0 * 10

def get_move_angle(co):
    #print(co)
    
    angle = (128-co)/128*30
    #print(angle)
    return int(angle)
    
camera = USBCamera(width=WIDTH, height=HEIGHT, capture_fps=30)
# camera = CSICamera(width=WIDTH, height=HEIGHT, capture_fps=30)
cc=100
while True:
    torch.cuda.current_stream().synchronize()
    st=time.time()
    image =camera.read()
   
    co=execute(image) 
    #send to mac
    str_co = ",".join(map(str, co))
    print(str_co)
    strco = str_co + "#"
    soc.send(strco.encode())
    
    cv2.imshow("ds", image)
    #print(co)
    #print(co[0][17][0])
    torch.cuda.current_stream().synchronize()
    end=time.time()
    cc = cc + 1
    if position == 0 or position == 1 or position == 2 or position == 3 or position == 4:
        pwm.ChangeDutyCycle(angleToDutyCycle(18+position*36))
        print("now angle", 18+position*36)
        cc=0
        zhongjianzhi = position
        position = 11
        
    if cc == 30:
        if len(co) != 0:
            if co[0][17][0] != 0:
                move_angle = get_move_angle(co[0][17][0])
                print("modify", move_angle)
                pwm.ChangeDutyCycle(angleToDutyCycle(18+zhongjianzhi*36+move_angle))
                position = 11
            if co[0][17][0] == 0:
                for i in range(17):
                    if co[0][i][0] != 0:
                        decider = co[0][i][0]
                        break
                if decider >=128:
                    move_angle = -30
                    print("modify", move_angle)
                    pwm.ChangeDutyCycle(angleToDutyCycle(18+zhongjianzhi*36+move_angle))
                    position = 11
                if decider <128:
                    move_angle = 30
                    print("modify", move_angle)
                    pwm.ChangeDutyCycle(angleToDutyCycle(18+zhongjianzhi*36+move_angle))
                    position = 11
        else: 
            print("no people here")
            #position = 10
    #if position == 10:
     #   pwm.ChangeDutyCycle(angleToDutyCycle(0))
       
    #print(end-st)
    if (cv2.waitKey(1) & 0xFF) == ord('q'):
        break
#camera.running = True
cap.release()
cv2.destroyAllWindows()

[[2, 95], [11, 80], [0, 0], [47, 100], [0, 0], [82, 174], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [32, 174]]
[[2, 97], [11, 82], [0, 0], [48, 101], [0, 0], [82, 174], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [30, 174]]
[[2, 98], [14, 82], [0, 0], [48, 101], [0, 0], [82, 175], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [30, 175]]
[[2, 101], [14, 84], [0, 0], [48, 98], [0, 0], [83, 175], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [32, 175]]
[[2, 102], [14, 85], [0, 0], [48, 100], [0, 0], [83, 175], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [32, 175]]
[[2, 106], [11, 89], [0, 0], [45, 100], [0, 0], [85, 174], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [35, 173]]
[[2, 106], [10, 92], [0, 0], [45, 101], [0, 0], [85, 175

[[9, 113], [22, 94], [2, 97], [52, 104], [0, 0], [87, 183], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [34, 179]]
[[9, 115], [22, 96], [2, 99], [50, 103], [0, 0], [86, 182], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [33, 178]]
[[8, 115], [21, 97], [2, 99], [48, 103], [0, 0], [86, 182], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [33, 179]]
[[4, 115], [21, 97], [2, 99], [48, 103], [0, 0], [85, 181], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [32, 178]]
[[4, 115], [19, 97], [0, 0], [48, 103], [0, 0], [85, 181], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [32, 178]]
[[4, 115], [21, 97], [2, 99], [49, 103], [0, 0], [86, 182], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [32, 178]]
[[5, 114], [21, 96], [2, 98], [49, 103], [0, 0]

[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [86, 179], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]

[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [86, 181], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [86, 182], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [87, 182], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [87, 182], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [87, 182], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]

[[0, 0], [0, 0], [0, 0], [29, 78], [0, 0], [86, 183], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], 

[[0, 0], [19, 95], [0, 0], [49, 102], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [18, 95], [0, 0], [48, 103], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [0, 0], [0, 0], [47, 106], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [0, 0], [0, 0], [47, 104], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [14, 94], [0, 0], [46, 102], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [13, 93], [0, 0], [45, 100], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [11, 94], [0, 0], [45, 102], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0


[[0, 0], [0, 0], [0, 0], [45, 105], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [0, 0], [0, 0], [42, 103], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]

[[0, 0], [0, 0], [0, 0], [42, 103], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]




[[0, 0], [0, 0], [0, 0], [45, 102], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]

[[0, 0], [0, 0], [0, 0], [45, 105], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [15, 98], [0, 0], [45, 102], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [14, 96], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 

...

[[2, 69], [17, 59], [2, 59], [40, 85], [0, 0], [74, 158], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [24, 162]]
[[2, 70], [17, 59], [2, 59], [41, 85], [0, 0], [74, 158], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [25, 163]]
[[2, 71], [18, 60], [2, 61], [42, 85], [0, 0], [74, 159], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [26, 162]]
[[3, 71], [19, 59], [2, 59], [44, 82], [0, 0], [77, 159], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [29, 161]]
[[9, 69], [24, 57], [2, 59], [48, 82], [0, 0], [79, 159], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [30, 162]]
[[13, 67], [29, 56], [4, 56], [52, 81], [0, 0], [82, 161], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [34, 162]]
[[20, 62], [34, 51], [14, 53], [58, 77], [0, 0], [86, 162

[[9, 113], [17, 91], [0, 0], [50, 88], [0, 0], [81, 166], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
[[9, 113], [17, 89], [0, 0], [49, 87], [0, 0], [81, 165], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [34, 166]]
[[9, 113], [17, 88], [0, 0], [49, 87], [0, 0], [81, 165], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [34, 166]]
[[12, 114], [17, 91], [0, 0], [49, 86], [0, 0], [81, 165], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [38, 169]]
[[12, 116], [17, 91], [0, 0], [49, 87], [0, 0], [81, 166], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
[[9, 109], [18, 87], [0, 0], [49, 87], [0, 0], [79, 167], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [32, 170]]
[[10, 106], [20, 87], [2, 91], [49, 87], [0, 0], [81, 167], [0

[[0, 0], [3, 105], [0, 0], [34, 105], [0, 0], [75, 169], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [30, 174]]
[[0, 0], [2, 105], [0, 0], [34, 105], [0, 0], [74, 169], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [27, 173]]
[[0, 0], [2, 106], [0, 0], [32, 107], [0, 0], [74, 169], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [31, 174]]
[[0, 0], [2, 106], [0, 0], [31, 107], [0, 0], [73, 170], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [30, 174]]
[[0, 0], [2, 106], [0, 0], [30, 107], [0, 0], [73, 170], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [30, 174]]
[[0, 0], [0, 0], [0, 0], [27, 108], [0, 0], [70, 170], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [30, 175]]
[[0, 0], [2, 106], [0, 0], [27, 109], [0, 0], [70, 167], [0, 0], [

[[0, 0], [0, 0], [0, 0], [29, 107], [0, 0], [69, 167], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [30, 174]],[[0, 0], [0, 0], [0, 0], [0, 0], [34, 105], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [0, 0], [0, 0], [29, 106], [0, 0], [69, 167], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [29, 174]]
[[0, 0], [2, 106], [0, 0], [30, 107], [0, 0], [69, 167], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [30, 174]]
[[0, 0], [2, 107], [0, 0], [30, 107], [0, 0], [69, 167], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [29, 174]]
[[0, 0], [2, 107], [0, 0], [30, 108], [0, 0], [70, 167], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [30, 174]]
[[0, 0], [2, 107], [0, 0], [30, 106], [0, 0], [70, 168], [0, 0], [0, 0], [0,

NameError: name 'cap' is not defined